In [ ]:
!pip install tokenizers==0.7.0

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1vwwwi-pUgyu6Xby0GtTirmarX40beplY',
                                    dest_path='./input.txt',
                                    overwrite=True)
gdd.download_file_from_google_drive(file_id='1PMMy92PnV8I6LjzLboUaZpBVbEJmDlL3',
                                    dest_path='./vocab.txt',
                                    overwrite=True)

In [ ]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    vocab_file=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##"
)

tokenizer.train(
    files=['vocab.txt'],
    limit_alphabet=1000,
    vocab_size=30000
)

tokenizer.save('.', 'temp')

In [ ]:
!git clone https://github.com/google-research/bert

In [ ]:
!python bert/create_pretraining_data.py --input_file=input.txt \
--output_file=tf_examples.tfrecord \
--vocab_file=temp-vocab.txt \
--do_lower_case=True \
--max_seq_length=120 \
--max_prediction_per_seq=20 \
--masked_lm_prob=0.15 \
--random_seed=42 \
--dupe_factor=5

In [ ]:
!python bert/run_pretraining.py \
  --input_file=tf_examples.tfrecord \
  --output_dir=/tmp/pretraining_output \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert/bert_config.json \
  --init_checkpoint=bert/bert_model.ckpt \
  --train_batch_size=32 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20 \
  --num_warmup_steps=10 \
  --learning_rate=2e-5

In [ ]:
!pip install tensorflow-gpu==1.14.0

In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
!cat bert_output.txt